# Features Ale

La idea de esta notebook es crear los siguientes features:

 - [Rango_horario](#Rango_horario)
 - [Prob_compra_x_usuario_x_dia](#Prob_compra_x_usuario_x_dia)

In [1]:
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

limpieza_path = os.path.join(current_folder, '..', 'Limpieza.ipynb')

In [3]:
%run $limpieza_path
df = get_clean_df()

In [4]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


#### Prob_compra_x_usuario_x_dia

In [5]:
df_eventos_aux = df.loc[df['event']=='conversion'].copy()

In [6]:
df_eventos_aux = df_eventos_aux[['event','timestamp','person']]

In [7]:
df_eventos_aux['dia'] = pd.to_datetime(df_eventos_aux['timestamp']).dt.weekday_name

In [8]:
df_eventos_aux['hora'] = pd.to_datetime(df_eventos_aux['timestamp']).dt.hour

In [9]:
df_compras_x_dia = df_eventos_aux.groupby(['person','dia']).size().reset_index(name='cant_compras_x_dia')

In [10]:
df_dias_c = df_eventos_aux.groupby(['person','event']).size().reset_index(name='cant_compras_totales')

In [11]:
Prob_compra_x_usuario_x_dia = pd.merge(df_compras_x_dia,df_dias_c,how="inner",on='person')

In [12]:
Prob_compra_x_usuario_x_dia['prob_compra_por_dia'] = Prob_compra_x_usuario_x_dia['cant_compras_x_dia'] / Prob_compra_x_usuario_x_dia['cant_compras_totales']

In [13]:
del Prob_compra_x_usuario_x_dia['cant_compras_x_dia']
del Prob_compra_x_usuario_x_dia['cant_compras_totales']
del Prob_compra_x_usuario_x_dia['event']

In [14]:
Prob_compra_x_usuario_x_dia = Prob_compra_x_usuario_x_dia.reset_index().groupby(['person', 'dia'])['prob_compra_por_dia'].aggregate('first').unstack()

In [15]:
Prob_compra_x_usuario_x_dia.fillna(0, inplace=True)

#### Rango_horario

In [16]:
Rango_horario = df_eventos_aux.copy()

In [17]:
Rango_horario['madrugada'] = np.where(((Rango_horario['hora']<7) & (Rango_horario['hora']>=0)), 1,0)
Rango_horario['maniana'] = np.where(((Rango_horario['hora']<13) & (Rango_horario['hora']>=7)), 1,0)
Rango_horario['tarde'] = np.where(((Rango_horario['hora']<19) & (Rango_horario['hora']>=13)), 1,0)
Rango_horario['noche'] = np.where(((Rango_horario['hora']<24) & (Rango_horario['hora']>=19)), 1,0)

In [18]:
del Rango_horario['dia']
del Rango_horario['event']
del Rango_horario['timestamp']
del Rango_horario['hora']

In [19]:
Rango_horario = Rango_horario.groupby('person').agg({'madrugada':'sum','maniana':'sum','tarde':'sum','noche':'sum'})

Se agregan los features al df_feature

In [20]:
df_features = df_features.merge(Prob_compra_x_usuario_x_dia, how='left', on='person')

In [21]:
df_features.fillna(0, inplace=True)

In [22]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [23]:
df_features = df_features.merge(Rango_horario, how='left', on='person')

In [24]:
assert(df_features.shape[0] == df['person'].unique().shape[0])